### What is Steganography?
Steganography is the practice of hiding secret data inside non-secret data. Most of the time, we're talking about hiding messages inside image files, but it doesn't have to be image files.
Let's say I want to share a secret recipe with you. I could pull up my favorite picture of you and me, encode the recipe into the picture, and post it on your Facebook wall or something. To the world, it looks like I just sent you a cool picture. But since you have the decoder, you run the image through it and now you know how to make my grandmother's world famous cookies.
Do you get what this is now? Good! Let's put it into practice.


### How does this work??
I'm glad you asked. First, you need a quick crash course in pixels. Your screen is made up of pixels, and probably quite a few of them. My monitor resolution says "1920x1080" which means my screen is 1920 pixels across and 1080 pixels deep, and 2,073,600 pixels altogether. Each one of those pixels is capable of emitting a light within a color range. The color of light it emits is (normally) based on a combination of the colors red green and blue. The intensity of each color can be from 0 to 255. In other words, RGB(255, 255, 255) is white, RGB(0, 0, 0) is black, and RGB(113, 238, 184) is seafoam green.
Image files are basically a serialization of an image's pixels and RGB values. The file tells the computer which pixels to light up and with which color. When we use the cv2 function imread and pass it an image file, that image file is translated into a numpy array containing the RGB value for each pixel in the image.

In [1]:
import cv2 as cv
import numpy as np
from skimage.io import imshow
from IPython.display import Image

In [2]:
def message2binary(message):
    if type(message) == str:
        result= ''.join([ format(ord(i), "08b") for i in message ])
    elif type(message) == bytes or type(message) == np.ndarray:
        result= [ format(i, "08b") for i in message ]
    elif type(message) == int or type(message) == np.uint8:
        result=format(message, "08b")
    elif type(message) == list:
        result= []
        result.append(format(message[0], "08b"))
        result.append(format(message[1], "08b"))
        result.append(format(message[2], "08b"))
        
    return result

In [3]:
def BinaryTointeger(binary):  
    binary1 = binary  
    decimal, i, n = 0, 0, 0
    while(binary != 0):  
        dec = binary % 10
        decimal = decimal + dec * pow(2, i)  
        binary = binary//10
        i += 1
    return (decimal)

In [4]:
def Binary2String(bin_data):
    str_data= ''
    for i in range(0, len(bin_data), 7): 
        temp_data = int(bin_data[i:i + 7]) 
        decimal_data = BinaryTointeger(temp_data) 
        str_data = str_data + chr(decimal_data)  
    return str_data

In [5]:
list1=[ ]
def encode_data(img, data):
    no_bytes=(img.shape[0] * img.shape[1] * 3) // 8
    #print("Maximum bytes to encode:", no_bytes)
    
    data+= '*****'    # Using as delimeter
    data_binary=message2binary(data)
    print(data_binary)
    data_len=len(data_binary)
    # print("The Length of Binary data",data_len)
    
    data_index = 0
    
    for i in img:
        for pixel in i:  
            r, g, b = message2binary(pixel)
            if data_index < data_len:
                pixel[0] = int(r[:-1] + data_binary[data_index], 2)
                data_index += 1
                list1.append(pixel[0])

            if data_index < data_len:
                pixel[1] = int(g[:-1] + data_binary[data_index], 2)
                data_index += 1
                list1.append(pixel[1])

            if data_index < data_len:
                pixel[2] = int(b[:-1] + data_binary[data_index], 2)
                data_index += 1
                list1.append(pixel[2])

            if data_index >= data_len:
                break
                
    cv.imwrite("final.png", img)
    print("\n\t\t ######### Encoded the saved as final.png #########")

In [6]:
img= cv.imread("images/m3.png")
img= cv.resize(img, (100,100))
# imshow(img)
encode_data(img, "MohammadRezaSajadi")

0100110101101111011010000110000101101101011011010110000101100100010100100110010101111010011000010101001101100001011010100110000101100100011010010010101000101010001010100010101000101010

		 ######### Encoded the saved as final.png #########


In [7]:
def decode_data(img):
    binary_data = ""
    for i in img:
        for pixel in i:
            r, g, b = message2binary(pixel) 
            binary_data += r[-1]
            binary_data += g[-1]
            binary_data += b[-1]
 
    all_bytes = [ binary_data[i: i+8] for i in range(0, len(binary_data), 8) ]
    decoded_data = ""
    for byte in all_bytes:
        decoded_data += chr(int(byte, 2))
        if decoded_data[-5:] == "*****": #Checking "*****"
            break

    print("The Decode data : ", decoded_data[:-5])

In [8]:
image1= cv.imread("final.png")
decode_data(image1)

The Decode data :  MohammadRezaSajadi
